In [20]:
from IPython.display import display, HTML
from IPython.display import clear_output

display(HTML("<style>.container { width:100% !important; }</style>"))

#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=True)

import warnings
warnings.filterwarnings('ignore')

In [21]:
import pandas as pd
import unicodedata
import re
import seaborn as sns
import time
import numpy as np
import math
import requests
import time
import datetime
#%pip install num2words
from num2words import num2words
from ast import literal_eval

In [23]:
games_metacritic = pd.read_pickle('final_version_metacritic.pkl')
games_vgchartz = pd.read_pickle('df_april_2023.pkl')

# Problématique

Comme on peut le voir sur la cellule suivante, on doit fusionner les deux dataframes en fonction de leur nom. Malheureusement, un simple merge ne suffira car il faut que les noms soient exactement similaires des deux cotés pour trouver correspondances.
On utilisera donc une librairie qui calcule la **distance de Levenshtein** (https://fr.wikipedia.org/wiki/Distance_de_Levenshtein) pour trouver un maximum de correspondances. On effectuera un préalable un nettoyage des chaines de caractères titre pour optimiser nos résultats.

In [26]:
display(games_metacritic.head())
display(games_vgchartz.head())

Name Platform Score Nbr_User_Critics  \
0  The Legend of Zelda: Ocarina of Time      N64    99             7922   
1              Tony Hawk's Pro Skater 2       PS    98              910   
2                   Grand Theft Auto IV      PS3    98             4975   
3                           SoulCalibur       DC    98              416   
4                   Grand Theft Auto IV     X360    98             3729   

  User_score  Year  Nbr_Awards  Release Date Critic_review_Positive  \
0        9.1  1998        16.0  Nov 23, 1998                     22   
1        7.5  2000        20.0  Sep 20, 2000                     19   
2        7.9  2008         4.0  Apr 29, 2008                     64   
3        8.5  1999        16.0  Sep  8, 1999                     24   
4        8.0  2008         4.0  Apr 29, 2008                     86   

  critic_review_mixed critic_review_negative user_review_positive  \
0                   0                      0                1,638   
1                   0                      0                   76   
2                   0                      0                  631   
3                   0                      0                   77   
4                   0                      0                  701   

  user_review_negative user_review_mixed  \
0                   95                91   
1                   19                13   
2                  112               163   
3                    2                 4   
4                  113               185   

                                            game_url  \
0  https://www.metacritic.com/game/nintendo-64/th...   
1  https://www.metacritic.com/game/playstation/to...   
2  https://www.metacritic.com/game/playstation-3/...   
3  https://www.metacritic.com/game/dreamcast/soul...   
4  https://www.metacritic.com/game/xbox-360/grand...   

                              Name_Platform_Year  \
0  The Legend of Zelda: Ocarina of Time N64 1998   
1               Tony Hawk's Pro Skater 2 PS 2000   
2                   Grand Theft Auto IV PS3 2008   
3                            SoulCalibur DC 1999   
4                  Grand Theft Auto IV X360 2008   

                           cleaned_name  
0  The Legend of Zelda: Ocarina of Time  
1              Tony Hawk's Pro Skater 2  
2                   Grand Theft Auto IV  
3                           SoulCalibur  
4                   Grand Theft Auto IV

title  total_sales         publisher  \
0                      Minecraft  238000000.0            Mojang   
1             Grand Theft Auto V  175000000.0    Rockstar Games   
2                     Wii Sports   82900000.0          Nintendo   
3  PlayerUnknown's Battlegrounds   70000000.0  PUBG Corporation   
4            Mario Kart 8 Deluxe   52000000.0          Nintendo   

          developer release_date platform  japan_sales  na_sales  other_sales  \
0         Mojang AB  18th Nov 11      All          0.0       0.0          0.0   
1    Rockstar North  17th Sep 13      All          0.0       0.0          0.0   
2      Nintendo EAD  19th Nov 06      Wii          0.0       0.0          0.0   
3  PUBG Corporation  17th Dec 17      All          0.0       0.0          0.0   
4      Nintendo EPD  28th Apr 17       NS          0.0       0.0          0.0   

   pal_sales  ...  vgchartz_score  critic_score  last_update  \
0        0.0  ...             NaN           NaN  08th Oct 20   
1        0.0  ...             NaN           NaN  08th Oct 20   
2        0.0  ...             NaN           7.7                
3        0.0  ...             NaN           NaN  24th Oct 20   
4        0.0  ...             NaN           9.3  19th Nov 18   

   id_platform_igdb    year  n_Results_found  total_sales_calculated  \
0               NaN  2011.0              2.0                     0.0   
1               NaN  2013.0              1.0                     0.0   
2               5.0  2006.0              1.0                     0.0   
3               NaN  2017.0              1.0                     0.0   
4             130.0  2017.0              1.0                     0.0   

   total_sales_diff  rank                   search_title  
0       238000000.0     1                      Minecraft  
1       175000000.0     2             Grand Theft Auto V  
2        82900000.0     3                     Wii Sports  
3        70000000.0     4  PlayerUnknown's Battlegrounds  
4        52000000.0     5            Mario Kart 8 Deluxe  

[5 rows x 22 columns]

# Fonctions de nettoyage

Ces fonctions va servent à faire le maximum pour faire ressembler les titres des jeux dans les deux dataframes.
Elles seront appliquées sur :
- games_vgchartz.search_title
- games_vgchartz.Name

In [28]:
def remove_accents(input_str):
    try:
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])
    except:
        print('error on this input string:', input_str)
        return input_str

In [29]:
remove_accents('Godé of wàr: Ragnarök')

'Gode of war: Ragnarok'

In [30]:
def remove_special_chars(input_str):
    try:
        return re.sub('\W+',' ', input_str).strip()
    except:
        print('error on this input string:', input_str)
        return input_str.strip()

In [31]:
def remove_dates_from_title(input_str):
    if '(' in input_str:
        if input_str.split('(')[1][:-1].isnumeric():  #ex: tomb raider (2013)
            return input_str.split('(')[0].strip()
    return input_str
remove_dates_from_title('tomb raider (2013)')

'tomb raider'

# Application des fonctions de nettoyage

In [52]:
# Colonnes numériques
cols_to_convert = ['Score', 'Nbr_User_Critics', 'User_score', 'Year', 'Critic_review_Positive', 'critic_review_mixed',
       'critic_review_negative', 'user_review_positive',
       'user_review_negative', 'user_review_mixed']
games_metacritic[cols_to_convert] = games_metacritic[cols_to_convert].replace(',', '', regex=True)
games_metacritic[cols_to_convert] = games_metacritic[cols_to_convert].apply(pd.to_numeric, errors='coerce')
games_metacritic[cols_to_convert] = games_metacritic[cols_to_convert].fillna(0)

In [34]:
# nettoyage des titres
games_metacritic['cleaned_name'] = games_metacritic['Name'].apply(remove_accents)
games_metacritic['cleaned_name'] = games_metacritic['cleaned_name'].apply(remove_dates_from_title)
games_metacritic['cleaned_name'] = games_metacritic['cleaned_name'].apply(remove_special_chars)
games_metacritic['cleaned_name'] = games_metacritic['cleaned_name'].apply(str.lower)
games_metacritic['Year'] = games_metacritic['Year'].astype('Int64').astype('str')

games_vgchartz['cleaned_name'] = games_vgchartz['search_title'].apply(remove_accents)
games_vgchartz['cleaned_name'] = games_vgchartz['cleaned_name'].apply(remove_dates_from_title)
games_vgchartz['cleaned_name'] = games_vgchartz['cleaned_name'].apply(remove_special_chars)
games_vgchartz['cleaned_name'] = games_vgchartz['cleaned_name'].apply(str.lower)

# Regroupement des jeux similaires

Puisque certains jeux sont regroupés par plateformes sur VGChartz (ceux ou Platform = 'All'), on fait de meme sur les jeux de metacritic dans un dataframe à part (**games_metacritic_grouped**)
Exemple sur les jeux 'Read dead Redemption'

In [87]:
games_metacritic[games_metacritic.cleaned_name.str.contains('red dead')][['Name', 'Platform', 'Year','cleaned_name']]

Name Platform    Year  \
19991                       Red Dead Redemption 2     XOne  2018.0   
20000                       Red Dead Redemption 2      PS4  2018.0   
20038                         Red Dead Redemption      PS3  2010.0   
20051                         Red Dead Redemption     X360  2010.0   
20143                       Red Dead Redemption 2       PC  2019.0   
21055  Red Dead Redemption: Undead Nightmare Pack      PS3  2010.0   
21135  Red Dead Redemption: Undead Nightmare Pack     X360  2010.0   
24458       Red Dead Redemption: Liars and Cheats      PS3  2010.0   
26898       Red Dead Redemption: Liars and Cheats     X360  2010.0   
29273                           Red Dead Revolver       XB  2004.0   
29471                           Red Dead Revolver      PS2  2004.0   

                                  cleaned_name_y  
19991                      red dead redemption 2  
20000                      red dead redemption 2  
20038                        red dead redemption  
20051                        red dead redemption  
20143                      red dead redemption 2  
21055  red dead redemption undead nightmare pack  
21135  red dead redemption undead nightmare pack  
24458       red dead redemption liars and cheats  
26898       red dead redemption liars and cheats  
29273                          red dead revolver  
29471                          red dead revolver

In [54]:
games_metacritic_grouped = games_metacritic.groupby(['cleaned_name', 'Year']).agg({
    'cleaned_name': 'max',
    'Year': 'max',
    'Score': 'mean',
    'Nbr_User_Critics': 'sum',
    'User_score': 'mean',
    'Nbr_Awards': 'sum',
    'Critic_review_Positive': 'sum',
    'critic_review_mixed': 'sum',
    'critic_review_negative': 'sum',
    'user_review_positive': 'sum',
    'user_review_negative': 'sum',
    'user_review_mixed': 'sum',
    'game_url': 'max'
})

games_metacritic_grouped = games_metacritic_grouped.sort_values(by='Score', ascending=False)

### Séparation du dataframe VGChartz

On sépare en fonction de l'appartenance à la plateforme 'All'.
On tentera de fusionner les jeux de cette manière :
- games_vgchartz_all  **et**  games_metacritic_grouped
- games_vgchartz  **et**  games_metacritic

In [69]:
games_vgchartz_all = games_vgchartz[games_vgchartz.platform == 'All']
games_vgchartz = games_vgchartz[games_vgchartz.platform != 'All']

In [70]:
games_metacritic["name_platform_year"] = games_metacritic['cleaned_name'] +" "+ games_metacritic['Platform'] +" "+ games_metacritic['Year'].astype(str)
games_vgchartz["name_platform_year"] = games_vgchartz['cleaned_name'] +" "+ games_vgchartz['platform'] +" "+ games_vgchartz['year'].astype(str)

games_vgchartz_all["name_year"] = games_vgchartz_all['cleaned_name'] +" "+ games_vgchartz_all['year'].astype(str)
games_metacritic_grouped["name_year"] = games_metacritic_grouped['cleaned_name'] +" "+ games_metacritic_grouped['Year'].astype(str)

# Basic merge :

On commence par un basic merge, puis on retire les jeux metacritic pour lesquels on a trouvé une correspondance pour **réduire la dimension** du dataframe sur lequel on fera un **fuzzy merge** qui est plus long à exécuter s'il y a beaucoup de lignes.

In [71]:
games_vgchartz_all_basic_merge = pd.merge(games_vgchartz_all, games_metacritic_grouped, how='inner', on='name_year')
games_vgchartz_all_tofind = pd.merge(games_vgchartz_all, games_metacritic_grouped, how='outer', on='name_year', indicator=True).query('_merge=="left_only"').drop(columns='_merge')
games_metacritic_grouped = pd.merge(games_vgchartz_all, games_metacritic_grouped, how='outer', on='name_year', indicator=True).query('_merge=="right_only"')

In [72]:
games_vgchartz_basic_merge = pd.merge(games_vgchartz, games_metacritic, how='inner', on='name_platform_year')
games_vgchartz_tofind = pd.merge(games_vgchartz, games_metacritic, how='outer', on='name_platform_year', indicator=True).query('_merge=="left_only"').drop(columns='_merge')
games_metacritic = pd.merge(games_vgchartz, games_metacritic, how='outer', on='name_platform_year', indicator=True).query('_merge=="right_only"')

In [73]:
games_basic_merge = pd.concat([games_vgchartz_all_basic_merge, games_vgchartz_basic_merge])
games_basic_merge = games_basic_merge[['title', 'total_sales', 'publisher', 'developer', 'release_date',
       'platform', 'japan_sales', 'na_sales', 'other_sales', 'pal_sales',
       'pos', 'user_score', 'vgchartz_score', 'critic_score', 'last_update',
       'id_platform_igdb', 'year', 'n_Results_found', 'total_sales_calculated',
       'total_sales_diff', 'rank', 'search_title', 'cleaned_name_x',
       'name_year', 'cleaned_name_y', 'Year', 'Score', 'Nbr_User_Critics',
       'User_score', 'Nbr_Awards', 'Critic_review_Positive',
       'critic_review_mixed', 'critic_review_negative', 'user_review_positive',
       'user_review_negative', 'user_review_mixed', 'game_url']]
games_basic_merge.to_pickle('games_basic_merge.pkl')

In [74]:
games_metacritic_grouped = games_metacritic_grouped[['name_year', 'cleaned_name_y', 'Year', 'Score', 'Nbr_User_Critics',
       'User_score', 'Nbr_Awards', 'Critic_review_Positive',
       'critic_review_mixed', 'critic_review_negative', 'user_review_positive',
       'user_review_negative', 'user_review_mixed', 'game_url']]

In [75]:
games_metacritic = games_metacritic[['name_platform_year', 'Name', 'Platform', 'Score', 'Nbr_User_Critics',
       'User_score', 'Year', 'Nbr_Awards', 'Release Date',
       'Critic_review_Positive', 'critic_review_mixed',
       'critic_review_negative', 'user_review_positive',
       'user_review_negative', 'user_review_mixed', 'game_url','cleaned_name_y']]

In [76]:
games_vgchartz_all_tofind = games_vgchartz_all_tofind[['title', 'total_sales', 'publisher', 'developer', 'release_date',
       'platform', 'japan_sales', 'na_sales', 'other_sales', 'pal_sales',
       'pos', 'user_score', 'vgchartz_score', 'critic_score', 'last_update',
       'id_platform_igdb', 'year', 'n_Results_found', 'total_sales_calculated',
       'total_sales_diff', 'rank', 'search_title', 'cleaned_name_x',
       'name_year']]

In [77]:
games_vgchartz_tofind = games_vgchartz_tofind[['title', 'total_sales', 'publisher', 'developer', 'release_date',
       'platform', 'japan_sales', 'na_sales', 'other_sales', 'pal_sales',
       'pos', 'user_score', 'vgchartz_score', 'critic_score', 'last_update',
       'id_platform_igdb', 'year', 'n_Results_found', 'total_sales_calculated',
       'total_sales_diff', 'rank', 'search_title', 'cleaned_name_x',
       'name_platform_year']]

# Fuzzy merge

La fonction fuzzy merge prendra en paramètre :
- names : Ca sera la liste des titres vgchartz
- names : Ca sera la liste des titres metacritic
- theshold : le score minimum à obtenir par la fonction pour considérer une correspondance

In [31]:
#%pip install fuzzywuzzy
from fuzzywuzzy import process
from datetime import datetime

def fuzzy_merge(names, names_2, threshold):
    print("start Time =", datetime.now().strftime("%H:%M:%S"))

    # make a dictionary of closest matches to names 
    i = 0
    keys = {}
    for name in names:
        i+=1
        print(i, ' /', len(names))
        #names in smaller dataset to compare and match#get closest match of `name` compared to larger data `member_names`    limit=1, 
        matching_name = ((process.extractOne(name, names_2, score_cutoff = threshold)))
        keys[name] = matching_name
        print(name, ':', matching_name, '\n')
        #you can limit to 1 with extractOne to take less time but I wanted to check if names were returning as correct.

        if i%50 == 0:
            clear_output(wait=True)  # Affichage de 7 résultats à la fois
        '''if i == 132:
            break'''

    print("end Time =", datetime.now().strftime("%H:%M:%S"))
    return keys

In [94]:
# Pour améliorer la performance des scripts (temps d'exécution), on filtre par date dans une boucle
matching_names_all = pd.DataFrame()

for year in games_vgchartz_all_tofind.year.unique():
    
    matching_names_tmp = fuzzy_merge(
        games_vgchartz_all_tofind[games_vgchartz_all_tofind.year == year].cleaned_name_x, 
        games_metacritic_grouped[games_metacritic_grouped.Year == year].cleaned_name_y, 
        90)
    
    matching_names_tmp = pd.DataFrame.from_dict(matching_names_tmp, orient='index')
    matching_names_all = pd.concat([matching_names_all, matching_names_tmp])
    
matching_names_all.to_pickle('matching_names_all.pkl')
print('done1')

start Time = 01:25:20
1  / 8
diablo iii : None 

2  / 8
tekken 7 : ('n', 90, 8206) 

3  / 8
monster hunter 4 : ('monster hunter 4 ultimate', 90, 7956) 

4  / 8
the room three : None 

5  / 8
elite dangerous horizons : ('elite dangerous', 90, 4444) 

6  / 8
60 seconds : None 

7  / 8
disgaea 5 : ('disgaea 5 alliance of vengeance', 90, 3839) 

8  / 8
atelier sophie the alchemist of the mysterious book : None 

end Time = 01:25:23
start Time = 01:25:23
1  / 2
zuma : None 

2  / 2
tomb raider the angel of darkness : ('lara croft tomb raider the angel of darkness', 95, 6896) 

end Time = 01:25:24
start Time = 01:25:24
1  / 8
fifa 11 : None 

2  / 8
zumba fitness : ('zumba fitness join the party', 90, 14514) 

3  / 8
monster hunter 3 : None 

4  / 8
red dead redemption undead nightmare : ('red dead redemption undead nightmare pack', 95, 9859) 

5  / 8
super street fighter iv arcade edition : ('super street fighter iv', 90, 11803) 

6  / 8
ben 10 ultimate alien cosmic destruction : None 

7  

deer hunter ii the hunt continues : None 

4  / 5
caesar iii : None 

5  / 5
china the forbidden city : None 

end Time = 01:25:51
start Time = 01:25:51
1  / 15
civilization ii : ('sid meier s civilization ii', 90, 10794) 

2  / 15
lords of the realm ii : None 

3  / 15
die hard trilogy : None 

4  / 15
formula 1 : None 

5  / 15
actua soccer : None 

6  / 15
broken sword the shadow of the templars : None 

7  / 15
leisure suit larry love for sail : None 

8  / 15
conquest of the new world : None 

9  / 15
barbie fashion designer : None 

10  / 15
blood omen legacy of kain : None 

11  / 15
shattered steel : None 

12  / 15
m a x mechanized assault exploration : None 

13  / 15
deadlock planetary conquest : None 

14  / 15
fox hunt : None 

15  / 15
allied general : None 

end Time = 01:25:51
start Time = 01:25:51
1  / 7
cuphead the delicious last course : ('cuphead in the delicious last course', 96, 3279) 

2  / 7
ebaseball powerful pro baseball 2022 : None 

3  / 7
earth defense forc

In [137]:
# Pour améliorer la performance des scripts (temps d'exécution), on filtre par plateforme et par date dans une boucle
matching_names_all = pd.DataFrame()

for year in games_vgchartz_tofind[games_vgchartz_tofind.year != '<NA>'].year.unique():
    for platform in games_vgchartz_tofind[games_vgchartz_tofind.year == year].platform.unique():
        matching_names_tmp = fuzzy_merge(
            games_vgchartz_tofind[(games_vgchartz_tofind.year == year) & (games_vgchartz_tofind.platform == platform)].cleaned_name_x, 
            games_metacritic[(games_metacritic.Year == year) & (games_metacritic.Platform == platform)].cleaned_name_y, 
            90)
        matching_names_tmp = pd.DataFrame.from_dict(matching_names_tmp, orient='index')
        matching_names_all = pd.concat([matching_names_all, matching_names_tmp])
    
matching_names_all.to_pickle('matching_names_1.pkl')
print('done1')

51  / 74
hidden invasion : None 

52  / 74
tokimeki memorial girl s side : None 

53  / 74
judie no atelier guramnat no renkinjutsushi : None 

54  / 74
nichibeikan pro yakyuu final league : None 

55  / 74
sky gunner : ('skygunner', 95, 24537) 

56  / 74
eikan wa kimini 2002 koshien no kodou : None 

57  / 74
the document of metal gear solid 2 : None 

58  / 74
next generation of chaos : None 

59  / 74
world fantasista : None 

60  / 74
woody woodpecker escape from buzz buzzard park : None 

61  / 74
jikkyou world soccer 2002 : None 

62  / 74
antz extreme racing : None 

63  / 74
dramatic soccer game nippon daihyou senshu ninarou : None 

64  / 74
konohana 2 todoke kanai requiem : None 

65  / 74
gacharoku : None 

66  / 74
runabout 3 neo age : None 

67  / 74
drummania 7th mix : None 

68  / 74
jitsumei jikkyou keiba dream classic 2002 : None 

69  / 74
roommate asami okusama wa joshikousei : None 

70  / 74
tsuki no hikari shizumeru kane no satsujin : None 

71  / 74
natsuiro no s

In [147]:
# games_vgchartz_tofind.cleaned_name_x                        games_metacritic.cleaned_name_y        
matching_names_all = pd.DataFrame()

games_vgchartz_tofind_na_year = games_vgchartz_tofind[games_vgchartz_tofind.year == '<NA>']

for platform in games_vgchartz_tofind_na_year.platform.unique():
    matching_names_tmp = fuzzy_merge(
        games_vgchartz_tofind_na_year[games_vgchartz_tofind_na_year.platform == platform].cleaned_name_x, 
        games_metacritic[games_metacritic.Platform == platform].cleaned_name_y, 
        90)
    matching_names_tmp = pd.DataFrame.from_dict(matching_names_tmp, orient='index')
    matching_names_all = pd.concat([matching_names_all, matching_names_tmp])
    
matching_names_all.to_pickle('matching_names_2.pkl')
print('done1')

401  / 448
wizorb : None 

402  / 448
ultimate general civil war : None 

403  / 448
redout enhanced edition : ('redout', 90, 22158) 

404  / 448
unholy heights : None 

405  / 448
super distro : ('er', 90, 29684) 

406  / 448
the chosen rpg : None 

407  / 448
pixeljunk monsters ultimate : ('pixeljunk monsters ultimate', 100, 26276) 

408  / 448
urban trial freestyle : ('urban trial freestyle', 100, 30110) 

409  / 448
galaxy on fire 2 full hd : ('galaxy on fire 2 full hd', 100, 29232) 

410  / 448
kings under the hill : None 

411  / 448
enemy mind : None 

412  / 448
anykey simulator : None 

413  / 448
immune true survival : None 

414  / 448
steel panthers : ('er', 90, 29684) 

415  / 448
tharsis : ('tharsis', 100, 29520) 

416  / 448
the secret of the nautilus : None 

417  / 448
between me and the night : ('between me and the night', 100, 29350) 

418  / 448
flying heroes : ('er', 90, 29684) 

419  / 448
record of lodoss war deedlit in wonder labyrinth : ('record of lodoss war d

# Série de traitements sur les dataframes matching_names

In [ ]:
matching_names_all = pd.read_pickle('matching_names_all.pkl')
matching_names_all['cleaned_name_x'] = matching_names_all.index
matching_names_all = matching_names_all.reset_index(drop=True)
matching_names_all['matching_name'] = matching_names_all[0]
matching_names_all['matching_score'] = matching_names_all[1]
matching_names_all['unknown_value'] = matching_names_all[2]
matching_names_all = matching_names_all.drop([0, 1, 2], axis=1)
matching_names_all = matching_names_all[matching_names_all['matching_name'].notna()]
tuple_data = {
    "cleaned_name_x": matching_names_all[matching_names_all['matching_score'].isna()]['cleaned_name_x'],
    "matching_name": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[0]),
    "matching_score": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[1]),
    "unknown_value": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[2])
}
tuple_data = pd.concat(tuple_data, axis = 1)

matching_names_all = pd.concat([matching_names_all[matching_names_all['matching_score'].notna()], tuple_data])
matching_names_all['matching_name_len'] = matching_names_all['matching_name'].str.len()
print(len(matching_names_all))
matching_names_all.to_pickle('matching_names_all.pkl')

In [ ]:
matching_names_all = pd.read_pickle('matching_names_1.pkl')
matching_names_all['cleaned_name_x'] = matching_names_all.index
matching_names_all = matching_names_all.reset_index(drop=True)
matching_names_all['matching_name'] = matching_names_all[0]
matching_names_all['matching_score'] = matching_names_all[1]
matching_names_all['unknown_value'] = matching_names_all[2]
matching_names_all = matching_names_all.drop([0, 1, 2], axis=1)
matching_names_all = matching_names_all[matching_names_all['matching_name'].notna()]
tuple_data = {
    "cleaned_name_x": matching_names_all[matching_names_all['matching_score'].isna()]['cleaned_name_x'],
    "matching_name": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[0]),
    "matching_score": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[1]),
    "unknown_value": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[2])
}
tuple_data = pd.concat(tuple_data, axis = 1)

matching_names_all = pd.concat([matching_names_all[matching_names_all['matching_score'].notna()], tuple_data])
matching_names_all['matching_name_len'] = matching_names_all['matching_name'].str.len()
print(len(matching_names_all))
matching_names_all.to_pickle('matching_names_1.pkl')

In [ ]:
matching_names_all = pd.read_pickle('matching_names_2.pkl') pd.read_pickle('matching_names_1.pkl')
matching_names_all['cleaned_name_x'] = matching_names_all.index
matching_names_all = matching_names_all.reset_index(drop=True)
matching_names_all['matching_name'] = matching_names_all[0]
matching_names_all['matching_score'] = matching_names_all[1]
matching_names_all['unknown_value'] = matching_names_all[2]
matching_names_all = matching_names_all.drop([0, 1, 2], axis=1)
matching_names_all = matching_names_all[matching_names_all['matching_name'].notna()]
tuple_data = {
    "cleaned_name_x": matching_names_all[matching_names_all['matching_score'].isna()]['cleaned_name_x'],
    "matching_name": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[0]),
    "matching_score": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[1]),
    "unknown_value": matching_names_all[matching_names_all['matching_score'].isna()]['matching_name'].apply(lambda x: literal_eval(str(x))[2])
}
tuple_data = pd.concat(tuple_data, axis = 1)

matching_names_all = pd.concat([matching_names_all[matching_names_all['matching_score'].notna()], tuple_data])
matching_names_all['matching_name_len'] = matching_names_all['matching_name'].str.len()
print(len(matching_names_all))
matching_names_all.to_pickle('matching_names_2.pkl')

# Résultats : 

Voici les correspondances obtenues grace aux scripts executés

In [89]:
display(pd.read_pickle('matching_names_1.pkl'))

cleaned_name_x  \
85                      pokemon ultra sun and ultra moon   
88     layton s mystery journey katrielle and the mil...   
90                                             minecraft   
118                          resident evil vii biohazard   
119        kingdom hearts hd ii 8 final chapter prologue   
...                                                  ...   
10636                dragon s lair 3d return to the lair   
10648             mx superfly featuring ricky carmichael   
10651                   deathrow underground team combat   
10652                dragon s lair 3d return to the lair   
10653                          zapper one wicked cricket   

                                           matching_name  matching_score  \
85                                     pokemon ultra sun            90.0   
88     layton s mystery journey katrielle and the mil...            99.0   
90                    minecraft new nintendo 3ds edition            90.0   
118                            resident evil 7 biohazard            92.0   
119         kingdom hearts hd 2 8 final chapter prologue            97.0   
...                                                  ...             ...   
10636                                   dragon s lair 3d            90.0   
10648                                        mx superfly            90.0   
10651                                           deathrow            90.0   
10652                                   dragon s lair 3d            90.0   
10653                                             zapper            90.0   

       unknown_value  matching_name_len  
85           20932.0                 17  
88           26271.0                 66  
90           29419.0                 34  
118          20647.0                 25  
119          24024.0                 44  
...              ...                ...  
10636        28634.0                 16  
10648        25137.0                 11  
10651        23085.0                  8  
10652        29415.0                 16  
10653        30464.0                  6  

[575 rows x 5 columns]

In [88]:
display(pd.read_pickle('matching_names_2.pkl'))

cleaned_name_x  \
0                         infestation survivor stories   
3                                      human fall flat   
5               dark souls ii scholar of the first sin   
11   pac man championship edition dx all you can ea...   
12                                        speedrunners   
..                                                 ...   
470                      one piece unlimited cruise sp   
475                   wrc fia world rally championship   
485                  pdc world championship darts 2008   
504                  pdc world championship darts 2008   
511                                            sumioni   

                              matching_name  matching_score  unknown_value  \
0                                  survivor            90.0        31560.0   
3                           human fall flat           100.0        26817.0   
5    dark souls ii scholar of the first sin           100.0        22775.0   
11          pac man championship edition dx            90.0        20539.0   
12                             speedrunners           100.0        21113.0   
..                                      ...             ...            ...   
470           one piece unlimited cruise sp           100.0        31185.0   
475        wrc fia world rally championship           100.0        27839.0   
485       pdc world championship darts 2008           100.0        31303.0   
504       pdc world championship darts 2009            97.0        31222.0   
511                      sumioni demon arts            90.0        30274.0   

     matching_name_len  
0                    8  
3                   15  
5                   38  
11                  31  
12                  12  
..                 ...  
470                 29  
475                 32  
485                 33  
504                 33  
511                 18  

[231 rows x 5 columns]

In [90]:
display(pd.read_pickle('matching_names_all.pkl'))

cleaned_name_x  \
49                          resident evil 7   
51                                     hack   
52                                     fate   
55                          civilization vi   
56                             va 11 hall a   
58                   dead or alive xtreme 3   
72                  resident evil 8 village   
73                      little nightmares 2   
95            lego star wars the video game   
96                          devil may cry 3   
98             oddworld stranger s wrath hd   
100                         dragon quest xi   
103                                 fifa 07   
104                     2006 fifa world cup   
212                         civilization ii   
227       cuphead the delicious last course   
1                                  tekken 7   
2                          monster hunter 4   
4                  elite dangerous horizons   
6                                 disgaea 5   
9         tomb raider the angel of darkness   
11                            zumba fitness   
13     red dead redemption undead nightmare   
14   super street fighter iv arcade edition   
16                   despicable me the game   
17                      clash of the titans   
44                     ducktales remastered   
64                      ufc 2009 undisputed   
68                                persona 4   
84       broken sword ii the smoking mirror   
111                     unreal tournament 3   
112                        colin mcrae dirt   
115                      dreamfall chapters   
159                 final fantasy xi online   
161                          uefa euro 2004   
235    marvel vs capcom 2 new age of heroes   
236                the emperor s new groove   
237                    dracula resurrection   

                                    matching_name  matching_score  \
49                      resident evil 7 biohazard            90.0   
51                           hack g u last recode            90.0   
52                   fate extella the umbral star            90.0   
55                    sid meier s civilization vi            90.0   
56        va 11 hall a cyberpunk bartender action            90.0   
58                 dead or alive xtreme 3 fortune            95.0   
72                          resident evil village            95.0   
73                           little nightmares ii            92.0   
95                                 lego star wars            90.0   
96              devil may cry 3 dante s awakening            90.0   
98                      oddworld stranger s wrath            95.0   
100      dragon quest xi echoes of an elusive age            90.0   
103                                fifa 07 soccer            90.0   
104                   fifa world cup germany 2006            95.0   
212                   sid meier s civilization ii            90.0   
227          cuphead in the delicious last course            96.0   
1                                               n            90.0   
2                       monster hunter 4 ultimate            90.0   
4                                 elite dangerous            90.0   
6                 disgaea 5 alliance of vengeance            90.0   
9    lara croft tomb raider the angel of darkness            95.0   
11                   zumba fitness join the party            90.0   
13      red dead redemption undead nightmare pack            95.0   
14                        super street fighter iv            90.0   
16           despicable me the game minion mayhem            90.0   
17              clash of the titans the videogame            90.0   
44                    disney ducktales remastered            95.0   
64                            ufc undisputed 2009            95.0   
68                   shin megami tensei persona 4            90.0   
84                broken sword the smoking mirror            95.0   
111                         unreal tournament iii       

# Fusion des jeux vgchartz et metacritic

In [587]:
matching_names = pd.concat([pd.read_pickle('matching_names_1.pkl'), pd.read_pickle('matching_names_2.pkl')])
matching_names = matching_names[matching_names.matching_name_len > 3]
matching_names = matching_names[['cleaned_name_x', 'matching_name']]

games_vgchartz_tofind = pd.merge(games_vgchartz_tofind, matching_names, on="cleaned_name_x", how='left')
games_vgchartz_tofind = games_vgchartz_tofind.drop_duplicates(subset=['rank'], keep='first')

In [588]:
fuzzy_merge_1 = pd.merge(games_vgchartz_tofind, games_metacritic, left_on="matching_name", right_on="cleaned_name_y", how='left').drop_duplicates(subset=['rank'], keep='first')

In [590]:
matching_names_all = pd.read_pickle('matching_names_all.pkl')
matching_names_all = matching_names_all[matching_names_all.matching_name_len > 3]
matching_names_all = matching_names_all[['cleaned_name_x', 'matching_name']]

games_vgchartz_all_tofind = pd.merge(games_vgchartz_all_tofind, matching_names_all, on="cleaned_name_x", how='left')

In [591]:
fuzzy_merge_2 = pd.merge(games_vgchartz_all_tofind, games_metacritic_grouped, left_on="matching_name", right_on="cleaned_name_y", how='left').drop_duplicates(subset=['rank'], keep='first')

In [592]:
print(len(fuzzy_merge_1))
print(len(fuzzy_merge_2))
print(len(games_basic_merge))
print(len(games_vgchartz_all_basic_merge))

11276
312
9403
695


In [523]:
print(len(pd.read_pickle('df_april_2023.pkl')))

20991


In [593]:
final = pd.concat([fuzzy_merge_1, fuzzy_merge_2, games_basic_merge, games_vgchartz_all_basic_merge]).drop_duplicates(subset=['rank'], keep='first')

In [594]:
# On garde juste les colonnes pertinentes
final = final[['title', 'total_sales', 'publisher', 'developer', 'release_date',
       'platform', 'japan_sales', 'na_sales', 'other_sales', 'pal_sales',
       'user_score', 'vgchartz_score', 'critic_score', 'last_update',
       'id_platform_igdb', 'year', 'n_Results_found', 'total_sales_calculated',
       'total_sales_diff', 'rank', 'search_title', 'Score', 'Nbr_User_Critics', 'User_score', 'Year',
       'Nbr_Awards', 'Release Date', 'Critic_review_Positive',
       'critic_review_mixed', 'critic_review_negative', 'user_review_positive',
       'user_review_negative', 'user_review_mixed', 'game_url']]

In [595]:
final['cleaned_name'] = final['search_title'].apply(remove_accents)
final['cleaned_name'] = final['cleaned_name'].apply(remove_dates_from_title)
final['cleaned_name'] = final['cleaned_name'].apply(remove_special_chars)
final['cleaned_name'] = final['cleaned_name'].apply(str.lower)

In [94]:
final = final.sort_values(by='rank', ascending=True).drop_duplicates(subset=['rank'], keep='first')

In [91]:
#final.to_pickle('final_merge_vgchartz_metacritic.pkl')
pd.read_pickle('final_merge_vgchartz_metacritic.pkl')

title  total_sales  \
0                              Minecraft  238000000.0   
1                     Grand Theft Auto V  175000000.0   
0                             Wii Sports   82900000.0   
2          PlayerUnknown's Battlegrounds   70000000.0   
0                    Mario Kart 8 Deluxe   52000000.0   
...                                  ...          ...   
11694       MX vs. ATV Supercross Encore      10000.0   
11695  Brain Exercise with Dr. Kawashima      10000.0   
11696         Brain Training for Dummies      10000.0   
11697            Bratz: Forever Diamondz      10000.0   
8707                  Rally Championship      10000.0   

                      publisher           developer release_date platform  \
0                        Mojang           Mojang AB  18th Nov 11      All   
1                Rockstar Games      Rockstar North  17th Sep 13      All   
0                      Nintendo        Nintendo EAD  19th Nov 06      Wii   
2              PUBG Corporation    PUBG Corporation  17th Dec 17      All   
0                      Nintendo        Nintendo EPD  28th Apr 17       NS   
...                         ...                 ...          ...      ...   
11694              Nordic Games     Rainbow Studios  27th Oct 15       PC   
11695              Namco Bandai  Namco Tales Studio  29th May 09       PC   
11696           Electronic Arts     Electronic Arts  21st Oct 08       PC   
11697                       THQ     Barking Lizards  18th Sep 06      GBA   
8707   Conspiracy Entertainment             Warthog  30th Jul 03       GC   

       japan_sales  na_sales  other_sales  pal_sales  ...  Nbr_Awards  \
0              0.0       0.0          0.0        0.0  ...         4.0   
1              0.0       0.0          0.0        0.0  ...         8.0   
0              0.0       0.0          0.0        0.0  ...         4.0   
2              0.0       0.0          0.0        0.0  ...         4.0   
0              0.0       0.0          0.0        0.0  ...         4.0   
...            ...       ...          ...        ...  ...         ...   
11694          0.0       0.0          0.0    10000.0  ...         NaN   
11695          0.0       0.0          0.0    10000.0  ...         NaN   
11696          0.0       0.0          0.0    10000.0  ...         NaN   
11697          0.0   10000.0          0.0        0.0  ...         NaN   
8707           0.0   10000.0          0.0        0.0  ...        17.0   

       Release Date  Critic_review_Positive critic_review_mixed  \
0               NaN                    33.0                 0.0   
1               NaN                   107.0                 1.0   
0               NaN                    31.0                20.0   
2               NaN                    41.0                 5.0   
0      Apr 28, 2017                    93.0                 2.0   
...             ...                     ...                 ...   
11694           NaN                     NaN                 NaN   
11695           NaN                     NaN                 NaN   
11696           NaN                     NaN                 NaN   
11697           NaN                     NaN                 NaN   
8707            NaN                     2.0                 6.0   

       critic_review_negative user_review_positive  user_review_negative  \
0                         0.0                1.458                 256.0   
1                         0.0             1317.000                 132.0   
0                         0.0              131.000                   4.0   
2                         0.0              114.000                 227.0   
0                         0.0              508.000                  53.0   
...                       ...                  ...                   ...   
11694                     NaN                  NaN                   NaN   
11695                     NaN                  NaN                   NaN   
11696                     NaN                  NaN         

# Fusion avec les jeux igdb.com

On fait une dernière fusion pour obtenir un dataframe contenant les données de :
- **VGchartz.com**
- **metacritic.com**
- **igdb.com**

In [655]:
y = pd.read_pickle('df_games_igdb.pkl') 

In [656]:
x = pd.read_pickle('final_merge_vgchartz_metacritic.pkl')

In [948]:
z = pd.merge(x, y, how='left', left_on='rank', right_on='pos')

In [982]:
z = z[['title', 'total_sales', 'publisher', 'developer', 'release_date',
       'platform', 'japan_sales', 'na_sales', 'other_sales', 'pal_sales',
       'id_platform_igdb', 'year', 'total_sales_calculated',
       'total_sales_diff', 'rank', 'Score', 'Nbr_User_Critics',
       'User_score', 'Nbr_Awards',
       'Critic_review_Positive', 'critic_review_mixed',
       'critic_review_negative', 'user_review_positive',
       'user_review_negative', 'user_review_mixed', 'game_url', 'cleaned_name',
       'id', 'category', 'first_release_date', 'genres', 'keywords', 'name',
       'tags', 'themes', 'age_ratings', 'aggregated_rating',
       'aggregated_rating_count', 'collection', 'involved_companies',
       'multiplayer_modes', 'rating', 'total_rating', 'total_rating_count', 'franchise']]

# Fonctions de nettoyage des dates 

In [876]:
def monthToNum(shortMonth):
    return {
            'Jan': '01',
            'Feb': '02',
            'Mar': '03',
            'Apr': '04',
            'May': '05',
            'Jun': '06',
            'Jul': '07',
            'Aug': '08',
            'Sep': '09', 
            'Oct': '10',
            'Nov': '11',
            'Dec': '12'
    }[shortMonth]

In [877]:
def get_day(input_date):
    try:
        return (input_date.split(' ')[0])[:-2]
    except:
        print('error date')
    #else :
    return None

In [878]:
def get_month(input_date):
    try:
        return monthToNum(input_date.split(' ')[1])
    except:
        print('error date')
    #else :
    return None

In [831]:
get_month("14th Mar 10")

'03'

In [949]:
z['len_date'] = z.release_date.str.len()

In [951]:
z[z.len_date != 11].release_date = np.nan

In [955]:
z.release_date.replace("", np.nan, inplace = True)

In [956]:
z['day'] = z['release_date'].apply(get_day)

error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date

In [957]:
z['month'] = z['release_date'].apply(get_month)

error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date
error date

In [958]:
z.release_date = z['year'].astype(str) + '-' + (z['month'])+ '-' + z['day']

In [959]:
z.first_release_date = (pd.to_datetime(z.first_release_date,unit='s'))

In [960]:
z.first_release_date = z.first_release_date.astype(str)

In [961]:
z.first_release_date.replace("NaT", np.nan, inplace = True)

In [962]:
z.release_date = z.first_release_date.fillna(z.release_date)

In [963]:
z.release_date = pd.to_datetime(z.release_date, format='%y-%m-%d', errors='ignore')

In [ ]:
z.release_date.astype(dtype = date, errors = 'ignore')

In [931]:
z[z.release_date.isna()]

title  total_sales  \
2817   PAC-MAN Championship Edition DX+ All You Can E...    1100000.0   
4387   Naruto Shippuden: Ultimate Ninja Storm 3 Full ...     670000.0   
5370                            Run for Money Tousouchuu     520000.0   
5434                                  Super Robot Wars Z     510000.0   
5497                       The Way of the Exploding Fist     500000.0   
6052              Super Robot Wars: Original Generations     450000.0   
6372                           Resident Evil HD Remaster     420000.0   
6825     Castlevania: Lords of Shadow - Ultimate Edition     380000.0   
7733                          Super Robot Wars OG Gaiden     310000.0   
8309   The Walking Dead - The Telltale Series: A New ...     280000.0   
8818                      Law & Order: Dead on the Money     250000.0   
9192      A grande bagunca espacial - The big space mess     230000.0   
9974                                            Nanoborg     200000.0   
10283                                    Magazime Editor     190000.0   
10617  Melissa K. and the Heart of Gold Collector's E...     180000.0   
10769        The Last Hope: Trump vs Mafia - North Korea     170000.0   
10818                 Daily Chthonicle: Editor's Edition     170000.0   
10860                       Splatter - Zombie Apocalypse     170000.0   
11347                        Football Manager Touch 2018     150000.0   
11535                         Tempest: Pirate Action RPG     150000.0   
12109                             Immune - True Survival     140000.0   
13730                        Gary Grigsby's World at War     100000.0   
13731          Jules Verne's Return To Mysterious Island     100000.0   
14107                                   Egg Monster Hero      90000.0   
15382   Mega Man Battle Network: Operation Shooting Star      70000.0   
16049                       Mobile Ops: The One Year War      60000.0   
16727                                       Star General      50000.0   
17732                                           Moonmist      30000.0   
17978                                Bikkuriman Daijiten      30000.0   
18093                   Treasures of the Savage Frontier      30000.0   
18336              Samurai Spirits: Tenkaichi Kenkakuden      30000.0   
18368                                 Softporn Adventure      30000.0   
18423                                       Snack Attack      30000.0   
18921                  PDC World Championship Darts 2008      20000.0   
18962                                       Zork Trilogy      20000.0   
19020                                       Dream Dancer      20000.0   
19064            Shonen Jump's Yu-Gi-Oh! GX Card Almanac      20000.0   
19395                                           Scarfman      20000.0   
19570              Carrier Strike: South Pacific 1942-44      20000.0   
19628                         Gary Grigsby's Pacific War      20000.0   
20019                       Agarest Senki: Re-appearance      10000.0   
20085                                     Galactic Chase      10000.0   
20126                                  Enchanter Trilogy      10000.0   
20611                                             Photar      10000.0   
20711                                         Pac Attack      10000.0   
20842                                      Chainz Galaxy      10000.0   
20939                          Brave: Shaman's Challenge      10000.0   

               publisher                            developer release_date  \
2817             Unknown                         Bandai Namco          NaN   
4387             Unknown                        CyberConnect2          NaN   
5370             Unknown                   Bandai Namco Games          NaN   
5434             Unknown                            Banpresto          NaN   
5497             Unknown             Beam Software Pty., Ltd.          NaN   
6052             Unknown                            Banpresto          NaN   


In [932]:
z['len_date'] = z.release_date.astype(str).str.len()

In [853]:
z[z.len_date != 10].release_date = np.nan

In [ ]:
pd.to_datetime(z.release_date, format='%Y-%m-%d')

In [855]:
z[z.len_date == 19].release_date = z[z.len_date == 19].release_date

title  total_sales  \
0                              Minecraft  238000000.0   
1                     Grand Theft Auto V  175000000.0   
2                             Wii Sports   82900000.0   
3          PlayerUnknown's Battlegrounds   70000000.0   
4                    Mario Kart 8 Deluxe   52000000.0   
...                                  ...          ...   
20984                     Bound By Flame      10000.0   
20986       MX vs. ATV Supercross Encore      10000.0   
20987  Brain Exercise with Dr. Kawashima      10000.0   
20989            Bratz: Forever Diamondz      10000.0   
20990                 Rally Championship      10000.0   

                      publisher           developer         release_date  \
0                        Mojang           Mojang AB  2011-11-18 00:00:00   
1                Rockstar Games      Rockstar North  2013-09-17 00:00:00   
2                      Nintendo        Nintendo EAD  2006-11-19 00:00:00   
3              PUBG Corporation    PUBG Corporation  2017-12-20 00:00:00   
4                      Nintendo        Nintendo EPD  2017-04-28 00:00:00   
...                         ...                 ...                  ...   
20984    Focus Home Interactive             Spiders  2014-05-06 00:00:00   
20986              Nordic Games     Rainbow Studios  2015-10-27 00:00:00   
20987              Namco Bandai  Namco Tales Studio  2009-05-29 00:00:00   
20989                       THQ     Barking Lizards  2006-09-01 00:00:00   
20990  Conspiracy Entertainment             Warthog  2002-05-31 00:00:00   

      platform  japan_sales  na_sales  other_sales  pal_sales  ...  \
0          All          0.0       0.0          0.0        0.0  ...   
1          All          0.0       0.0          0.0        0.0  ...   
2          Wii          0.0       0.0          0.0        0.0  ...   
3          All          0.0       0.0          0.0        0.0  ...   
4           NS          0.0       0.0          0.0        0.0  ...   
...        ...          ...       ...          ...        ...  ...   
20984     X360          0.0       0.0          0.0    10000.0  ...   
20986       PC          0.0       0.0          0.0    10000.0  ...   
20987       PC          0.0       0.0          0.0    10000.0  ...   
20989      GBA          0.0   10000.0          0.0        0.0  ...   
20990       GC          0.0   10000.0          0.0        0.0  ...   

           multiplayer_modes     rating  total_rating total_rating_count  \
0      [11137, 11138, 11139]  84.466621     84.411882             1482.0   
1       [1045, 12794, 12795]  90.189546     89.163739             3897.0   
2                    [14699]  74.577763     74.788881              293.0   
3       [6128, 11050, 11051]  73.782971     74.422736              356.0   
4                    [17123]  84.228666     88.526098              401.0   
...                      ...        ...           ...                ...   
20984                    NaN  62.969737     60.166687               48.0   
20986                 [8167]  50.000000     35.000000                4.0   
20987                    NaN        NaN     48.666667                3.0   
20989                    NaN        NaN           NaN                NaN   
20990                    NaN        NaN     50.000000                1.0   

           pos franchise  len_date  day  month              new_col  
0          1.0       NaN        19   18     11  2011-11-18 00:00:00  
1          2.0       NaN        19   17     09  2013-09-17 00:00:00  
2          3.0       NaN        19   19     11  2006-11-19 00:00:00  
3          4.0       NaN        19   17     12  2017-12-20 00:00:00  
4          5.0       NaN        19   28     04  2017-04-28 00:00:00  
...        ...       ...       ...  ...    ...                  ...  
20984  20985.0       NaN        19   07     05  2014-05-06 00:00:00  
20986  20987.0       NaN        19   27     10  2015-10-27 00:00:00  
20987  20988.0       NaN        19   29     05  2009-

In [859]:
z.release_date = z.release_date.str.replace(' 00:00:00', '')

In [879]:
(pd.to_datetime(z.first_release_date,unit='s'))

0       2011-11-18
1       2013-09-17
2       2006-11-19
3       2017-12-20
4       2017-04-28
           ...    
20986   2015-10-27
20987   2009-05-29
20988          NaT
20989   2006-09-01
20990   2002-05-31
Name: first_release_date, Length: 20991, dtype: datetime64[ns]

In [673]:
z[(z.vgchartz_score.notna()) & (z.Score.isna()) & (z.aggregated_rating.isna())]

title  total_sales  \
134             Pokémon Heart Gold / Soul Silver Version   12720000.0   
1317              Super Mario All-Stars: Limited Edition    2240000.0   
2345                            MAG: Massive Action Game    1320000.0   
2398                          LEGO Batman: The Videogame    1300000.0   
4271                                       Madden NFL 13     700000.0   
4324           Kirby's Dream Collection: Special Edition     690000.0   
4593                      Cooking Mama 4: Kitchen Magic!     630000.0   
4888                                     NEKOPARA Vol. 1     580000.0   
6350                            Just Dance: Summer Party     420000.0   
8216                      Disney Channel: All Star Party     280000.0   
9015                                       FIFA Football     240000.0   
9048                     The Price Is Right 2010 Edition     240000.0   
10027                                           Dwarfs!?     200000.0   
11318                          NCAA Football 09 All-Play     160000.0   
11564                            Record of Agarest War 2     150000.0   
11949  Holy Invasion of Privacy, Badman! What Did I D...     140000.0   
12660                    Yu-Gi-Oh! 5D's Wheelie Breakers     120000.0   
12991  The Great Ace Attorney 2 - The Resolve of Ryuu...     120000.0   
13180  Jake Hunter Detective Story: Memories of the Past     110000.0   
13570                           America's Next Top Model     100000.0   
14033                             Farming Simulator 2011     100000.0   
15526                 Burnout Paradise: The Ultimate Box      70000.0   
15724                                         Death Mark      60000.0   
17882                Classic Dungeon: Fuyoku no Masoujin      30000.0   
18066                         Mary Skelter: Nightmares 2      30000.0   
18256                              The Longest 5 Minutes      30000.0   
19751             Castlevania: Lords of Shadow - Reverie      20000.0   

                         publisher                  developer release_date  \
134                       Nintendo                 Game Freak  14th Mar 10   
1317                      Nintendo               Nintendo EAD  12th Dec 10   
2345   Sony Computer Entertainment         Zipper Interactive  26th Jan 10   
2398      Warner Bros. Interactive          Traveller's Tales  23rd Sep 08   
4271                     EA Sports                 EA Tiburon  28th Aug 12   
4324                      Nintendo             HAL Laboratory  16th Sep 12   
4593                       Majesco          Cooking Mama Ltd.  15th Nov 11   
4888                       Unknown                 Neko Works                
6350                       Ubisoft              Ubisoft Paris  19th Jul 11   
8216    Disney Interactive Studios       Page 44 Studios, LLC  26th Oct 10   
9015                     EA Sports                  EA Canada  27th Sep 11   
9048                       Ubisoft                 Ludia Inc.  22nd Sep 09   
10027         Tripwire Interactive                 Power of 2  04th May 11   
11318                    EA Sports                Team Fusion  15th Jul 08   
11564                  Aksys Games              Compile Heart  26th Jun 12   
11949                  NIS America                    Acquire  16th Jul 09   
12660                       Konami                     Konami  19th May 09   
12991                       Capcom                     Capcom  03rd Aug 17   
13180                  Aksys Games                    WorkJam  26th May 09   
13570     Conspiracy Entertainment  Independent Arts Software  01st Jun 10   
14033                      Unknown            Giants Software  29th Oct 10   
15526              Electronic Arts            Criterion Games  06th Feb 09   
15724              Experience Inc.            Experience Inc.  01st Jun 17   
17882         Nippon Ichi Software              System Prisma  18th Feb 10   
18066                Compile Heart              Compile 

In [976]:
z[(z.game_url.notna()) & (z.id.notna())]

title  total_sales                 publisher  \
0                          Minecraft  238000000.0                    Mojang   
1                 Grand Theft Auto V  175000000.0            Rockstar Games   
2                         Wii Sports   82900000.0                  Nintendo   
3      PlayerUnknown's Battlegrounds   70000000.0          PUBG Corporation   
4                Mario Kart 8 Deluxe   52000000.0                  Nintendo   
...                              ...          ...                       ...   
20966                      Bomberman      10000.0      Hudson Entertainment   
20974    Madagascar: Escape 2 Africa      10000.0                Activision   
20980                Boulder Dash-XL      10000.0                   Kalypso   
20983                          Trine      10000.0                   Nobilis   
20990             Rally Championship      10000.0  Conspiracy Entertainment   

              developer release_date platform  japan_sales  na_sales  \
0             Mojang AB   2011-11-18      All          0.0       0.0   
1        Rockstar North   2013-09-17      All          0.0       0.0   
2          Nintendo EAD   2006-11-19      Wii          0.0       0.0   
3      PUBG Corporation   2017-12-20      All          0.0       0.0   
4          Nintendo EPD   2017-04-28       NS          0.0       0.0   
...                 ...          ...      ...          ...       ...   
20966       Hudson Soft   2008-09-12      Wii      10000.0       0.0   
20974             Aspyr   2008-11-04       PC          0.0   10000.0   
20980      Catnip Games   2011-07-13       PC          0.0       0.0   
20983        Frozenbyte   2009-07-03       PC          0.0       0.0   
20990           Warthog   2002-05-31       GC          0.0   10000.0   

       other_sales  pal_sales  ...  \
0              0.0        0.0  ...   
1              0.0        0.0  ...   
2              0.0        0.0  ...   
3              0.0        0.0  ...   
4              0.0        0.0  ...   
...            ...        ...  ...   
20966          0.0        0.0  ...   
20974          0.0        0.0  ...   
20980          0.0    10000.0  ...   
20983          0.0    10000.0  ...   
20990          0.0        0.0  ...   

                                             age_ratings aggregated_rating  \
0      [31660, 46780, 47365, 47366, 47368, 47370, 109...         84.357143   
1      [45948, 47891, 47892, 47893, 47894, 47895, 111...         88.137931   
2        [189, 36304, 48968, 62413, 62414, 62415, 62416]         75.000000   
3                          [31701, 31702, 91665, 113558]         75.062500   
4      [12328, 12329, 68115, 74139, 85481, 85482, 86513]         92.823529   
...                                                  ...               ...   
20966                                            [66594]               NaN   
20974                                     [27507, 71056]         45.000000   
20980                                     [16462, 72629]               NaN   
20983                                     [42268, 64938]         85.428571   
20990                                             [2111]         50.000000   

       aggregated_rating_count  collection             involved_companies  \
0                         17.0      1130.0                       [102771]   
1                         37.0       847.0          [18070, 18071, 20018]   
2                          3.0       681.0                  [4658, 22251]   
3                         21.0         NaN  [55299, 58862, 58863, 163990]   
4                         20.0       449.0          [43580, 43581, 46347]   
...                        ...         ...                            ...   
20966                      NaN         NaN                       [167987]   
20974                      4.0      3140.0      [9741, 9743, 9744, 76616]   
20980                      NaN      1756.0                            NaN   
20983                      9.0       887.0           

In [980]:
my_dict = {}

for col in z.columns:
    my_dict[col] = col

In [983]:
my_dict = {'title': 'title',
 'total_sales': 'total_sales',
 'publisher': 'publisher',
 'developer': 'developer',
 'release_date': 'release_date',
 'platform': 'platform',
 'japan_sales': 'japan_sales',
 'na_sales': 'na_sales',
 'other_sales': 'other_sales',
 'pal_sales': 'pal_sales',
 'id_platform_igdb': 'id_platform_igdb',
 'year': 'year',
 'total_sales_calculated': 'total_sales_calculated',
 'total_sales_diff': 'total_sales_diff',
 'rank': 'rank',
 'Score': 'meta_score',
 'Nbr_User_Critics': 'meta_nb_user_critics',
 'User_score': 'meta_user_score',
 'Nbr_Awards': 'meta_nb_awards',
 'Critic_review_Positive': 'meta_critic_review_positive',
 'critic_review_mixed': 'meta_critic_review_mixed',
 'critic_review_negative': 'meta_critic_review_negative',
 'user_review_positive': 'meta_user_review_positive',
 'user_review_negative': 'meta_user_review_negative',
 'user_review_mixed': 'meta_user_review_mixed',
 'game_url': 'meta_game_url',
 'cleaned_name': 'cleaned_name',
 'id': 'id_igdb_game',
 'category': 'category',
 'first_release_date': 'first_release_date',
 'genres': 'genres',
 'keywords': 'keywords',
 'name': 'igdb_name',
 'tags': 'tags',
 'themes': 'themes',
 'age_ratings': 'age_ratings',
 'aggregated_rating': 'igdb_aggregated_rating',
 'aggregated_rating_count': 'igdb_aggregated_rating_count',
 'collection': 'collection',
 'involved_companies': 'involved_companies',
 'multiplayer_modes': 'multiplayer_modes',
 'rating': 'igdb_rating',
 'total_rating': 'igdb_total_rating',
 'total_rating_count': 'igdb_total_rating_count',
 'franchise': 'franchise'}

In [984]:
z.rename(columns=my_dict, inplace=True)

In [987]:
z.drop('Release Date', axis=1, inplace = True)

In [80]:
z

title  total_sales  \
0                              Minecraft  238000000.0   
1                     Grand Theft Auto V  175000000.0   
2                             Wii Sports   82900000.0   
3          PlayerUnknown's Battlegrounds   70000000.0   
4                    Mario Kart 8 Deluxe   52000000.0   
...                                  ...          ...   
20986       MX vs. ATV Supercross Encore      10000.0   
20987  Brain Exercise with Dr. Kawashima      10000.0   
20988         Brain Training for Dummies      10000.0   
20989            Bratz: Forever Diamondz      10000.0   
20990                 Rally Championship      10000.0   

                      publisher           developer release_date platform  \
0                        Mojang           Mojang AB   2011-11-18      All   
1                Rockstar Games      Rockstar North   2013-09-17      All   
2                      Nintendo        Nintendo EAD   2006-11-19      Wii   
3              PUBG Corporation    PUBG Corporation   2017-12-20      All   
4                      Nintendo        Nintendo EPD   2017-04-28       NS   
...                         ...                 ...          ...      ...   
20986              Nordic Games     Rainbow Studios   2015-10-27       PC   
20987              Namco Bandai  Namco Tales Studio   2009-05-29       PC   
20988           Electronic Arts     Electronic Arts   2008-10-21       PC   
20989                       THQ     Barking Lizards   2006-09-01      GBA   
20990  Conspiracy Entertainment             Warthog   2002-05-31       GC   

       japan_sales  na_sales  other_sales  pal_sales  ...  \
0              0.0       0.0          0.0        0.0  ...   
1              0.0       0.0          0.0        0.0  ...   
2              0.0       0.0          0.0        0.0  ...   
3              0.0       0.0          0.0        0.0  ...   
4              0.0       0.0          0.0        0.0  ...   
...            ...       ...          ...        ...  ...   
20986          0.0       0.0          0.0    10000.0  ...   
20987          0.0       0.0          0.0    10000.0  ...   
20988          0.0       0.0          0.0    10000.0  ...   
20989          0.0   10000.0          0.0        0.0  ...   
20990          0.0   10000.0          0.0        0.0  ...   

                                             age_ratings  \
0      [31660, 46780, 47365, 47366, 47368, 47370, 109...   
1      [45948, 47891, 47892, 47893, 47894, 47895, 111...   
2        [189, 36304, 48968, 62413, 62414, 62415, 62416]   
3                          [31701, 31702, 91665, 113558]   
4      [12328, 12329, 68115, 74139, 85481, 85482, 86513]   
...                                                  ...   
20986                            [23192, 115374, 115375]   
20987                                             [9862]   
20988                                                NaN   
20989                                   [137712, 137713]   
20990                                             [2111]   

      igdb_aggregated_rating  igdb_aggregated_rating_count  collection  \
0                  84.357143                          17.0      1130.0   
1                  88.137931                          37.0       847.0   
2                  75.000000                           3.0       681.0   
3                  75.062500                          21.0         NaN   
4                  92.823529                          20.0       449.0   
...                      ...                           ...         ...   
20986              20.000000                           2.0       746.0   
20987              48.666667                           3.0         NaN   
20988                    NaN                           NaN         NaN   
20989                    NaN                           NaN         NaN   
20990              50.000000                           1.0         NaN   

                  involved_companies      multiplayer_modes  igdb_rating  \
0     

In [989]:
z.to_pickle('vgchartz_games_merge_metacritic_igdb.pkl')